In [ ]:
import os
import numpy as np
import pywt
from scipy.stats import entropy
import pandas as pd
import mne

def hjorth_parameters(signal):
    """Compute Hjorth parameters: activity, mobility, complexity."""
    activity = np.var(signal)
    diff_signal = np.diff(signal)
    mobility = np.sqrt(np.var(diff_signal) / activity)
    diff2_signal = np.diff(diff_signal)
    complexity = np.sqrt(np.var(diff2_signal) / np.var(diff_signal))
    return activity, mobility, complexity

def bandpower_ratio(fft, freqs):
    """Calculate bandpower ratios."""
    delta_power = np.sum(np.abs(fft[(freqs >= 0.5) & (freqs < 4)])**2)
    theta_power = np.sum(np.abs(fft[(freqs >= 4) & (freqs < 8)])**2)
    alpha_power = np.sum(np.abs(fft[(freqs >= 8) & (freqs < 13)])**2)
    beta_power = np.sum(np.abs(fft[(freqs >= 13) & (freqs < 30)])**2)

    delta_theta_ratio = delta_power / theta_power if theta_power != 0 else 0

    return delta_power, theta_power, alpha_power, beta_power, delta_theta_ratio

def wavelet_features(signal, wavelet='sym8', level=5):
    """Extract wavelet-based features."""
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    features = []

    for i, sub_band in enumerate(coeffs):
        # Energy
        energy = np.sum(sub_band**2)

        # Entropy
        hist, bin_edges = np.histogram(sub_band, bins='auto', density=True)
        pdf = hist / np.sum(hist)
        entropy_value = -np.sum(pdf * np.log2(pdf + np.finfo(float).eps))

        # Standard deviation, variance, and mean of absolute values
        std_dev = np.std(np.abs(sub_band))
        variance = np.var(np.abs(sub_band))
        mean_abs = np.mean(np.abs(sub_band))

        features.extend([energy, entropy_value, std_dev, variance, mean_abs])

    return features

def process_txt_features(input_dir, output_dir, wavelet='sym8', level=5):
    """Process a single TXT file from the Bonn University dataset, 
    extract features from all ICA components into one row, and append to a CSV file.
    """
    
    try:
        # Read the TXT file
        data = np.loadtxt(input_dir)

        # Create a raw MNE object (assuming sampling frequency of 173.61 Hz)
        ch_names = ['EEG']
        ch_types = ['eeg']
        info = mne.create_info(ch_names=ch_names, sfreq=173.61, ch_types=ch_types)  # Adjust sfreq if needed
        raw = mne.io.RawArray(data.reshape(1, -1), info)
        raw.filter(1, None, method='iir')

        # Apply ICA
        ica = mne.preprocessing.ICA(n_components=4, max_iter='auto', random_state=0)
        ica.fit(raw)

        # Extract features for each ICA component and concatenate into a single row
        all_features = []
        for i in range(ica.n_components_):
            ica_data = ica.get_sources(raw).get_data(picks=[i])
            avg_signal = ica_data.flatten()

            # Frequency domain features (FFT)
            fft = np.fft.fft(avg_signal)
            freqs = np.fft.fftfreq(len(avg_signal), d=1/raw.info['sfreq'])
            delta_power, theta_power, alpha_power, beta_power, delta_theta_ratio = bandpower_ratio(fft, freqs)

            # Hjorth parameters
            activity, mobility, complexity = hjorth_parameters(avg_signal)

            # Time-domain features
            power = np.mean(avg_signal ** 2)  
            signal_range = np.max(avg_signal) - np.min(avg_signal) 

            # Calculate duration of the signal
            duration = raw.times[-1]  

            # Wavelet Features (using level 5)
            wavelet_feat = wavelet_features(avg_signal, wavelet='sym8', level=5)

            # Add features to the list
            all_features.extend([power, signal_range, delta_power, theta_power, 
                                 alpha_power, beta_power, delta_theta_ratio, 
                                 activity, mobility, complexity, duration])
            all_features.extend(wavelet_feat)

        # Prepare data for saving, ensuring 'status' is the last column
        data = {f'feature_{i}': [val] for i, val in enumerate(all_features)}
        
        # --- Assign label based on filename ---
        if "Z" in filename:
            data['status'] = ['epilepsy']  # Example: files with 'Z' are epilepsy
        else:
            data['status'] = ['healthy']  # Example: other files are healthy
        # -------------------------------------

        # Convert to DataFrame and save
        df = pd.DataFrame(data)

        if os.path.exists(output_dir):
            df.to_csv(output_dir, mode='a', header=False, index=False)
        else:
            df.to_csv(output_dir, mode='w', header=True, index=False)

    except Exception as e:
        print(f"Error processing {input_dir}: {e}")
        print("Skipping this file and continuing...")
        return

# Main script
path = "f/F/"  # Replace with your directory
output_dir = "bonn_features.csv"  # Replace with your desired output file

# Loop through all TXT files in the directory
for filename in os.listdir(path):
    if filename.endswith('.txt'):
        input_dir = os.path.join(path, filename)
        
        # Process the file and extract features
        process_txt_features(input_dir, output_dir)
        
        print(f"Processed: {input_dir}")

In [3]:
import pywt
import numpy as np
from scipy.stats import entropy, skew, kurtosis
from scipy.signal import hilbert, welch, periodogram
import pandas as pd
import os
import emd  # Make sure to install EMD-signal: pip install EMD-signal

# --- Helper Functions ---
def hjorth_parameters(signal):
    first_deriv = np.diff(signal)
    second_deriv = np.diff(first_deriv)

    var_signal = np.var(signal)
    var_first_deriv = np.var(first_deriv)
    var_second_deriv = np.var(second_deriv)

    activity = var_signal
    mobility = np.sqrt(var_first_deriv / var_signal)
    complexity = np.sqrt(var_second_deriv / var_first_deriv) / mobility

    return activity, mobility, complexity

def wavelet_features(signal, wavelet='db4', level=5):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    features = []
    for coeff in coeffs:
        energy = np.sum(coeff ** 2)
        hist, bin_edges = np.histogram(coeff, bins='auto', density=True)
        pdf = hist * np.diff(bin_edges)
        ent = entropy(pdf)
        std_dev = np.std(np.abs(coeff))
        variance = np.var(coeff)
        mean_abs = np.mean(np.abs(coeff))
        features.append([energy, ent, std_dev, variance, mean_abs])
    return features

def hurst_exponent(signal):
    n = len(signal)
    t = np.arange(1, n + 1)
    mean_t = np.cumsum(signal - np.mean(signal))
    rs = (np.max(mean_t) - np.min(mean_t)) / np.std(signal)
    hurst = np.log(rs) / np.log(n)
    return hurst

def fractal_dimension(signal, kmax=10):
    L = []
    for k in range(1, kmax):
        n = len(signal) // k
        reduced_signal = signal[:n * k].reshape((k, n)).mean(axis=1)
        L.append(np.sum(np.abs(np.diff(reduced_signal))) / n)
    try:
        fd = -np.polyfit(np.log(range(1, kmax)), np.log(L), 1)[0]
    except:
        fd = np.nan
    return fd

def signal_energy(signal):
    return np.sum(np.square(signal)) / len(signal)

def root_mean_square(signal):
    return np.sqrt(np.mean(np.square(signal)))

def amplitude_envelope_correlation(signal, sfreq):
    analytic_signal = hilbert(signal)
    amplitude_envelope = np.abs(analytic_signal)
    return np.corrcoef(amplitude_envelope[:-1], amplitude_envelope[1:])[0, 1]

def spectral_band_features(signal, sfreq):
    eeg_bands = {'Delta': (0.5, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}

    freqs, psd = welch(signal, sfreq, nperseg=256)
    band_power = {band: np.mean(psd[(freqs >= fmin) & (freqs < fmax)])
                  for band, (fmin, fmax) in eeg_bands.items()}
    return band_power

def spectral_entropy(signal, sfreq, method='welch'):
    freqs, psd = welch(signal, sfreq) if method == 'welch' else periodogram(signal, sfreq)
    psd_norm = np.divide(psd, psd.sum())
    se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
    return se

def spectral_edge_frequency(signal, sfreq, edge=0.95):
    freqs, psd = welch(signal, sfreq)
    power = np.cumsum(psd)
    total_power = power[-1]
    sef = freqs[np.where(power >= edge * total_power)[0][0]]
    return sef

def zero_crossing_rate(signal):
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    return len(zero_crossings) / len(signal)

def signal_range(signal):
    return np.max(signal) - np.min(signal)

def emd_features(signal):
    imfs = emd.sift.sift(signal)
    features = []
    for imf in imfs:
        inst_freq = np.diff(np.unwrap(np.angle(hilbert(imf)))) / (2 * np.pi)
        energy = np.sum(imf ** 2)
        features.extend([np.mean(inst_freq), np.std(inst_freq), energy])
    return features

# --- Main Processing Function ---
def process_txt_features_improved(txt_file_path, output_csv_path, sfreq=250):
    try:
        signal = np.loadtxt(txt_file_path)
    except Exception as e:
        print(f"Skipping file due to error in reading: {txt_file_path}\nError: {e}")
        return

    signal = (signal - np.mean(signal)) / np.std(signal)

    # Feature extraction
    activity, mobility, complexity = hjorth_parameters(signal)
    wavelet_feats = wavelet_features(signal)
    hurst = hurst_exponent(signal)
    fd = fractal_dimension(signal)
    emd_feats = emd_features(signal)
    energy = signal_energy(signal)
    rms = root_mean_square(signal)
    spectral_feat = spectral_band_features(signal, sfreq)
    total_power = sum(spectral_feat.values())
    relative_band_power = {f"rel_{band}": power / total_power for band, power in spectral_feat.items()}
    spec_entropy = spectral_entropy(signal, sfreq)
    sef = spectral_edge_frequency(signal, sfreq)
    amp_env_corr = amplitude_envelope_correlation(signal, sfreq)
    sig_mean = np.mean(signal)
    sig_var = np.var(signal)
    sig_skewness = skew(signal)
    sig_kurt = kurtosis(signal)
    zc_rate = zero_crossing_rate(signal)
    sig_range = signal_range(signal)

    data = {
        # 'signal_energy': [energy],
        # 'rms': [rms],
        'spectral_entropy': [spec_entropy],
        'sef': [sef],
        'amplitude_env_corr': [amp_env_corr],
        'mean': [sig_mean],
        # 'variance': [sig_var],
        'skewness': [sig_skewness],
        'kurtosis': [sig_kurt],
        'zero_crossing_rate': [zc_rate],
        'range': [sig_range],
        'hurst_exponent': [hurst],
        # 'fractal_dimension': [fd],
        # 'hjorth_activity': [activity],
        'hjorth_mobility': [mobility],
        'hjorth_complexity': [complexity]
    }

    data.update(spectral_feat)
    data.update(relative_band_power)

    for i, sub_band_feats in enumerate(wavelet_feats):
        for j, feat in enumerate(sub_band_feats):
            data[f"wavelet_sb{i}_feat{j}"] = [feat]

    data.update({'emd_feat_' + str(i): [feat] for i, feat in enumerate(emd_feats)})

    df = pd.DataFrame(data)
    df['status'] = 'no_epilepsy'

    if os.path.exists(output_csv_path):
        df.to_csv(output_csv_path, mode='a', header=False, index=False)
    else:
        df.to_csv(output_csv_path, mode='w', header=True, index=False)

# --- Process Multiple Files ---
input_folder = "o/O/"
output_csv = "1221.csv"

for txt_file in os.listdir(input_folder):
    if txt_file.endswith(".txt"):
        process_txt_features_improved(os.path.join(input_folder, txt_file), output_csv)


C:\Users\PC\AppData\Local\Temp\ipykernel_24092\2477291893.py:53: RuntimeWarning: divide by zero encountered in log
  fd = -np.polyfit(np.log(range(1, kmax)), np.log(L), 1)[0]
C:\Users\PC\AppData\Local\Temp\ipykernel_24092\2477291893.py:53: RuntimeWarning: divide by zero encountered in log
  fd = -np.polyfit(np.log(range(1, kmax)), np.log(L), 1)[0]
C:\Users\PC\AppData\Local\Temp\ipykernel_24092\2477291893.py:53: RuntimeWarning: divide by zero encountered in log
  fd = -np.polyfit(np.log(range(1, kmax)), np.log(L), 1)[0]
C:\Users\PC\AppData\Local\Temp\ipykernel_24092\2477291893.py:53: RuntimeWarning: divide by zero encountered in log
  fd = -np.polyfit(np.log(range(1, kmax)), np.log(L), 1)[0]
C:\Users\PC\AppData\Local\Temp\ipykernel_24092\2477291893.py:53: RuntimeWarning: divide by zero encountered in log
  fd = -np.polyfit(np.log(range(1, kmax)), np.log(L), 1)[0]
C:\Users\PC\AppData\Local\Temp\ipykernel_24092\2477291893.py:53: RuntimeWarning: divide by zero encountered in log
  fd = -np

In [46]:
import pywt
import numpy as np
import pandas as pd
import os
from scipy.stats import kurtosis
import mne

# --- Helper Functions ---
def hjorth_parameters(signal):
    first_deriv = np.diff(signal)
    second_deriv = np.diff(first_deriv)

    var_signal = np.var(signal)
    var_first_deriv = np.var(first_deriv)
    var_second_deriv = np.var(second_deriv)

    mobility = np.sqrt(var_first_deriv / var_signal)
    complexity = np.sqrt(var_second_deriv / var_first_deriv) / mobility

    return mobility, complexity

def wavelet_features(signal, wavelet='db4', level=5, max_features=26):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    features = []
    for coeff in coeffs:
        features.append(np.mean(coeff))  # Mean of coefficients
        features.append(np.std(coeff))  # Std deviation
        features.append(np.max(coeff))  # Max value
        if len(features) >= max_features:
            break
    return features[:max_features]

# --- Main Processing Function ---
def process_txt_features_with_headers(txt_file_path, output_csv_path, sfreq=250):
    try:
        signal = np.loadtxt(txt_file_path)
    except Exception as e:
        print(f"Skipping file due to error in reading: {txt_file_path}\nError: {e}")
        return

    # Create an MNE RawArray object with correct channel name
    info = mne.create_info(ch_names=['EEG'], sfreq=sfreq) 
    raw = mne.io.RawArray(signal.reshape(1, -1), info)

    # Apply low-pass filter 
    raw.filter(None, 40, picks='EEG') 

    # Extract the filtered signal
    signal = raw.get_data()[0]

    # Normalize signal
    signal = (signal - np.mean(signal)) / np.std(signal)

    # Feature extraction
    sig_mean = np.mean(signal)
    sig_kurtosis = kurtosis(signal)
    hjorth_mobility, hjorth_complexity = hjorth_parameters(signal)
    wavelet_feats = wavelet_features(signal)

    # Combine features
    features = [
        sig_mean, sig_kurtosis, hjorth_mobility, hjorth_complexity, 
        *wavelet_feats                                              
    ]

    # Feature names
    feature_names = [
        "Mean", "Kurtosis", "Hjorth_Mobility", "Hjorth_Complexity"
    ] + [f"Wavelet_Feature_{i+1}" for i in range(len(wavelet_feats))]

    # Ensure no more than 30 features
    features = features[:30]
    feature_names = feature_names[:30]

    # Create DataFrame
    df = pd.DataFrame([features], columns=feature_names)
    df['Status'] = 'no_epilepsy' 

    # Save to CSV
    if os.path.exists(output_csv_path):
        df.to_csv(output_csv_path, mode='a', header=False, index=False)
    else:
        df.to_csv(output_csv_path, mode='w', header=True, index=False)

# --- Process Multiple Files ---
input_folder = "z/Z/" 
output_csv = "z_s.csv"

for txt_file in os.listdir(input_folder):
    if txt_file.endswith(".txt"):
        process_txt_features_with_headers(os.path.join(input_folder, txt_file), output_csv)

In [11]:
import pywt
import numpy as np
import pandas as pd
import os
from scipy.stats import kurtosis, skew
import mne

# --- Helper Functions ---
def hjorth_parameters(signal):
    first_deriv = np.diff(signal)
    second_deriv = np.diff(first_deriv)

    var_signal = np.var(signal)
    var_first_deriv = np.var(first_deriv)
    var_second_deriv = np.var(second_deriv)

    mobility = np.sqrt(var_first_deriv / var_signal)
    complexity = np.sqrt(var_second_deriv / var_first_deriv) / mobility

    return mobility, complexity

def wavelet_features(signal, wavelet='db4', level=5, max_features=26):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    features = []
    for i, coeff in enumerate(coeffs):
        features.append(np.mean(coeff))      # Mean of coefficients
        features.append(np.std(coeff))       # Std deviation
        features.append(np.max(coeff))       # Max value
        features.append(np.sum(coeff**2))    # Energy
        features.append(np.sum(np.abs(coeff)))  # Sum of absolute values

        # Relative energy 
        total_energy = np.sum([np.sum(c**2) for c in coeffs])
        features.append(np.sum(coeff**2) / total_energy) 

        # Relative amplitude
        total_amplitude = np.sum([np.sum(np.abs(c)) for c in coeffs])
        features.append(np.sum(np.abs(coeff)) / total_amplitude)

        # Fluctuation index
        features.append(np.std(coeff) / np.mean(coeff))

        # Coefficient of variation
        features.append(np.std(coeff) / np.mean(np.abs(coeff)))

        # Skewness and Kurtosis
        features.append(skew(coeff))
        features.append(kurtosis(coeff))

        if i > 0:  # Calculate ratio of absolute mean values of adjacent sub-bands
            features.append(np.abs(np.mean(coeffs[i-1])) / np.abs(np.mean(coeff)))

        if len(features) >= max_features:
            break
    return features[:max_features]

# --- Main Processing Function ---
def process_txt_features_with_headers(txt_file_path, output_csv_path, sfreq=250):
    try:
        signal = np.loadtxt(txt_file_path)
    except Exception as e:
        print(f"Skipping file due to error in reading: {txt_file_path}\nError: {e}")
        return

    # Create an MNE RawArray object with correct channel name
    info = mne.create_info(ch_names=['EEG'], sfreq=sfreq) 
    raw = mne.io.RawArray(signal.reshape(1, -1), info)

    # Apply low-pass filter 
    raw.filter(None, 40, picks='EEG') 

    # Extract the filtered signal
    signal = raw.get_data()[0]

    # Normalize signal
    signal = (signal - np.mean(signal)) / np.std(signal)

    # Feature extraction
    sig_mean = np.mean(signal)
    sig_kurtosis = kurtosis(signal)
    hjorth_mobility, hjorth_complexity = hjorth_parameters(signal)
    wavelet_feats = wavelet_features(signal, max_features=60)  # Increased max_features

    # Combine features
    features = [
        sig_mean, sig_kurtosis, hjorth_mobility, hjorth_complexity, 
        *wavelet_feats                                          
    ]

    # Feature names (update this with more descriptive names)
    feature_names = [
        "Mean", "Kurtosis", "Hjorth_Mobility", "Hjorth_Complexity"
    ] + [f"Wavelet_Feature_{i+1}" for i in range(len(wavelet_feats))]

    # Ensure no more than 60 features (adjust as needed)
    features = features[:60]
    feature_names = feature_names[:60]

    # Create DataFrame
    df = pd.DataFrame([features], columns=feature_names)
    df['Status'] = 'interictal' 

    # Save to CSV
    if os.path.exists(output_csv_path):
        df.to_csv(output_csv_path, mode='a', header=False, index=False)
    else:
        df.to_csv(output_csv_path, mode='w', header=True, index=False)

# --- Process Multiple Files ---
input_folder = "f/F/" 
output_csv = "mitdiec.csv"

for txt_file in os.listdir(input_folder):
    if txt_file.endswith(".txt"):
        process_txt_features_with_headers(os.path.join(input_folder, txt_file), output_csv)

Creating RawArray with float64 data, n_channels=1, n_times=4097
    Range : 0 ... 4096 =      0.000 ...    16.384 secs
Ready.
No data channels found. The highpass and lowpass values in the measurement info will not be updated.
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 83 samples (0.332 s)

Creating RawArray with float64 data, n_channels=1, n_times=4097
    Range : 0 ... 4096 =      0.000 ...    16.384 secs
Ready.
No data channels found. The highpass and lowpass values in the measurement info will not be updated.
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 40 Hz

FIR filter par

In [37]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Load your CSV file
file_path = 'simsim.csv'  # Replace with your CSV file path
data = pd.read_csv(file_path)

# Separate features (X) and labels (y) if available
X = data.iloc[:, :-1]  # Assuming the last column is the label
y = data.iloc[:, -1]   # Optional: Use if you need the labels later

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=0.95) # Retain 95% of the variance
X_pca = pca.fit_transform(X_scaled)

# Print the selected components
print("PCA Components Shape:", X_pca.shape)
print("Explained Variance Ratio:", pca.explained_variance_ratio_)


PCA Components Shape: (100, 3)
Explained Variance Ratio: [0.51523951 0.24764916 0.19268054]


In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Load your CSV file
file_path = 'simsim.csv'  # Replace with your CSV file path
data = pd.read_csv(file_path)

# Separate features (X) and labels (y) 
X = data.iloc[:, :-1]  # Assuming the last column is the label
y = data.iloc[:, -1]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA with enough components to cover the original feature count
pca = PCA(n_components=X.shape[1])  # Keep all components initially
X_pca = pca.fit_transform(X_scaled)

# Calculate the explained variance ratio for each component
explained_variance_ratio = pca.explained_variance_ratio_

# Set a threshold for "extremely good" components
threshold = 0.8  # You can adjust this value

# Find the indices of components with explained variance ratio above the threshold
good_components_indices = [i for i, ratio in enumerate(explained_variance_ratio) if ratio > threshold]

# If no components meet the threshold, take the one with the highest explained variance
if not good_components_indices:
    good_components_indices = [explained_variance_ratio.argmax()]

# Get the most important feature for each "extremely good" component
most_important_features_indices = abs(pca.components_[good_components_indices]).argmax(axis=1)

# Select the columns corresponding to the most important features from the ORIGINAL data
X_reduced = X.iloc[:, most_important_features_indices]

# Add the label column back to the reduced dataframe
X_reduced['label'] = y

# Save the reduced data to a new CSV file
output_file_path = 'reduced_simsim.csv'  # Choose a name for your output file
X_reduced.to_csv(output_file_path, index=False)

print(f"Reduced data saved to {output_file_path}")